# Challenge 3

In [1]:
%load_ext autotime

## Preprocessing

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image, ImageOps
import os
from imutils import paths
%matplotlib inline

time: 836 ms


In [3]:
def processImage(im, desired_size, debug=False):
    # get old size
    old_size = im.size 
    # detect the ratio to which we need to scale the image
    ratio = float(desired_size)/max(old_size)
    # detect the new size of the image to fit the square box
    new_size = tuple([int(x*ratio) for x in old_size])
    # resize to new size
    im = im.resize(new_size, Image.ANTIALIAS)
    # define background color
    bgcolor = 0xffffff
    # create new blank image and "paste" the resized image in the middle of it
    new_im = Image.new("RGB", (desired_size, desired_size), bgcolor)
    new_im.paste(im, ((desired_size-new_size[0])//2,
                    (desired_size-new_size[1])//2))
    
    # do color equalization
    arr = Image.fromarray(np.uint8(new_im))
    new_im = ImageOps.equalize(arr)
    return new_im

time: 18.7 ms


In [4]:
def load(imagePaths, verbose = -1):
    data = []
    labels = []

    #loop over input images
    for (i, imagePath) in enumerate(imagePaths):
        # assumes our path has the following format:
        # /path/to/dataset/{class}/{image}.jpg
        image = Image.open(imagePath)
        label = imagePath.split(os.path.sep)[-2]
        
        #preprocess
        image = processImage(image, 128)
        
        data.append(np.asarray(image, dtype=np.uint8) / 255)
        labels.append(label)

        # show an update every 'verbose' images
        if verbose > 0 and i > 0 and (i+1) % verbose == 0:
            print('[INFO] processed {}/{}'.format(i + 1, len(imagePaths)))
        
    return (np.array(data), np.array(labels))

time: 18.2 ms


In [5]:
imagePaths = list(paths.list_images('gear_images'))
(data, labels) = load(imagePaths, verbose=100)

size = 128 * 128 * 3
print(data.shape)
print(labels.shape)

[INFO] processed 100/2122
[INFO] processed 200/2122
[INFO] processed 300/2122
[INFO] processed 400/2122
[INFO] processed 500/2122
[INFO] processed 600/2122
[INFO] processed 700/2122
[INFO] processed 800/2122
[INFO] processed 900/2122
[INFO] processed 1000/2122
[INFO] processed 1100/2122
[INFO] processed 1200/2122
[INFO] processed 1300/2122
[INFO] processed 1400/2122
[INFO] processed 1500/2122
[INFO] processed 1600/2122
[INFO] processed 1700/2122
[INFO] processed 1800/2122
[INFO] processed 1900/2122
[INFO] processed 2000/2122
[INFO] processed 2100/2122
(2122, 128, 128, 3)
(2122,)
time: 11.4 s


## Modelling

### Encode Labels

In [6]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
encoded_labels = lb.fit_transform(labels)

print(lb.classes_)
print(encoded_labels)

['axes' 'boots' 'carabiners' 'crampons' 'gloves' 'hardshell_jackets'
 'harnesses' 'helmets' 'insulated_jackets' 'pulleys' 'rope' 'tents']
[[0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]
time: 30.8 ms


### Split Data

In [7]:
from sklearn.model_selection import train_test_split

data_train, data_test, label_train, label_test = train_test_split(data, encoded_labels, test_size=0.3, random_state=1)

print(data_train.shape)
print(label_train.shape)
print(data_test.shape)
print(label_test.shape)

(1485, 128, 128, 3)
(1485, 12)
(637, 128, 128, 3)
(637, 12)
time: 323 ms


### Train model

#### Pipeline

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

num_classes = 12

model = Sequential()
model.add(Conv2D(32, kernel_size=(7, 7), activation='relu', input_shape=data_train.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


time: 1.31 s


#### Compiler

In [9]:
# For a multi-class classification problem
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

time: 38.3 ms


#### Train

In [10]:
# Train the model, iterating on the data in batches of 32 samples
model.fit(data_train, label_train, epochs=10, batch_size=32, validation_data=(data_test, label_test))

Train on 1485 samples, validate on 637 samples
Epoch 1/10
1485/1485 [==============================] - 53s 36ms/step - loss: 8.5889 - acc: 0.2283 - val_loss: 1.9348 - val_acc: 0.3626
Epoch 2/10
1485/1485 [==============================] - 52s 35ms/step - loss: 1.2645 - acc: 0.6067 - val_loss: 0.8113 - val_acc: 0.7425
Epoch 3/10
1485/1485 [==============================] - 52s 35ms/step - loss: 0.6039 - acc: 0.8283 - val_loss: 0.6162 - val_acc: 0.7849
Epoch 4/10
1485/1485 [==============================] - 52s 35ms/step - loss: 0.2964 - acc: 0.9024 - val_loss: 0.8333 - val_acc: 0.7708
Epoch 5/10
1485/1485 [==============================] - 52s 35ms/step - loss: 0.1458 - acc: 0.9502 - val_loss: 0.4593 - val_acc: 0.8901
Epoch 6/10
1485/1485 [==============================] - 52s 35ms/step - loss: 0.0817 - acc: 0.9690 - val_loss: 0.3796 - val_acc: 0.9278
Epoch 7/10
1485/1485 [==============================] - 52s 35ms/step - loss: 0.0780 - acc: 0.9744 - val_loss: 0.5551 - val_acc: 0.9058
E

time: 8min 43s


In [11]:
# Convert labels to categorical one-hot encoding
#one_hot_labels = keras.utils.to_categorical(labels, num_classes=10)

# Train the model, iterating on the data in batches of 32 samples
#model.fit(data, one_hot_labels, epochs=10, batch_size=32)

time: 1.68 ms


#### Predict

In [14]:
pred = model.predict(data_test, batch_size = 100)

time: 7.1 s


In [16]:
print(pred)
print(label_test)

[[3.7412898e-13 1.5312960e-20 7.3370934e-15 ... 1.1816918e-21
  1.5455980e-20 8.1173856e-25]
 [1.5979819e-13 2.0329386e-26 1.0000000e+00 ... 1.7349518e-15
  6.6115247e-20 5.4963124e-25]
 [2.0272275e-06 3.9511156e-06 5.2762981e-11 ... 4.2275011e-07
  9.9969816e-01 4.4549434e-11]
 ...
 [7.7011490e-01 2.5834581e-11 1.1339691e-16 ... 2.9962848e-16
  1.1316949e-03 1.5276756e-08]
 [1.3657031e-13 6.0343265e-19 1.6733420e-17 ... 1.5280472e-18
  1.0000000e+00 2.2148904e-25]
 [2.4626794e-04 2.4998961e-12 3.5562389e-06 ... 5.9731580e-09
  5.0068462e-01 2.7841113e-19]]
[[0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 1 0 0]]
time: 3.09 ms


### Score model

In [12]:
score = model.evaluate(data_test, label_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.5345606900217214
Test accuracy: 0.9372056516785076
time: 7.12 s



### Full Report

In [17]:
import sklearn.metrics as metrics

target_names = lb.classes_
print(metrics.classification_report(label_test.argmax(axis=1), pred.argmax(axis=1), target_names=target_names))

                   precision    recall  f1-score   support

             axes       0.84      0.84      0.84        19
            boots       1.00      0.97      0.99        37
       carabiners       0.99      0.93      0.96        82
         crampons       0.89      0.92      0.90        36
           gloves       0.90      0.88      0.89        65
hardshell_jackets       0.96      1.00      0.98       129
        harnesses       0.93      0.98      0.96        56
          helmets       0.97      0.91      0.94        33
insulated_jackets       0.98      0.91      0.95        70
          pulleys       0.80      0.75      0.77        16
             rope       0.85      0.98      0.91        58
            tents       0.97      0.89      0.93        36

      avg / total       0.94      0.94      0.94       637

time: 4.2 ms


### Save model

In [18]:
model.save("keras-cnn.h5")

time: 125 ms
